# Ingest Election candidates

### Config

In [15]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [16]:
remote_path = 'http://pacha.datawheel.us/'
local_path = '../data/'

### Imports

In [17]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

In [18]:
download_zip_file(remote_path, local_path, 'politics3.zip')
extract_zip_file(local_path, 'temp.zip')

Downloading... http://pacha.datawheel.us/politics3.zip
Unzipping... ../data/temp.zip


True

In [19]:
df = download_file(remote_path,local_path,'2018-02-08_candidatos_id.csv')
df = df.rename(columns={'candidato_id':'candidate_id','candidato':'candidate'})
df = df.astype({'candidate_id':'int'})
list(df)

Already downloaded. Using: ../data/2018-02-08_candidatos_id.csv


['candidate_id', 'candidate']

### Ingest

In [20]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'politics', 'dim_candidates')

DROP TABLE IF EXISTS politics.dim_candidates;
CREATE TABLE "politics"."dim_candidates" (
"candidate_id" INTEGER,
  "candidate" TEXT
)
COPY "politics"."dim_candidates" ("candidate_id","candidate") FROM STDIN WITH CSV HEADER DELIMITER ',';
